In [6]:
# TO-DO:

# Improve model and gain insights via Evaluation metrics
# Host on AWS
# Improve conversational answers
# Course codes in source.

In [8]:
# Imports
import os
import uuid

from dotenv import load_dotenv

import pinecone
from pinecone import Pinecone, ServerlessSpec

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

In [10]:
load_dotenv()  # Loading Environment

True

In [12]:
spec = ServerlessSpec(cloud="aws", region="us-east-1")  # spec instance
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))  # pinecone object

In [14]:
# Accessing Indices on pinecone
index_name = "courses-ds"
index = pc.Index(index_name)

# embedding model
embedding_model = OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY"))

In [15]:
# function to retrieve relevant chunks from pinecone vector database
def retrieve_from_pinecone(
    query, top_k=10
):  # top_k indicates how many chunks we want to retrive from database
    query_embedding = embedding_model.embed_query(query)
    results = index.query(
        vector=[query_embedding], top_k=top_k, include_metadata=True
    )  # querying database

    relevant_chunks = [match["metadata"].get("text") for match in results["matches"]]

    # Returning relevant chunks
    return relevant_chunks

In [16]:
# llm instance
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.3,  # Experiment with different Temperatures
)

In [20]:
# Variable storing how many messages (10) will be remembered (Human + Bot): For conversationalness
memory_messages = 10  # Update to change context lengths


# Converting query to a prompt that has context from the documents
def query_to_prompt(query, state_messages):
    relevant_chunks = retrieve_from_pinecone(
        query
    )  # retrieveing relevant chunks to make context
    context = "\n\n -------------- \n\n".join(chunk for chunk in relevant_chunks)
    context = f"\n\nCONTEXT: {context}"

    # Play around with this.
    sys_message_template = (
        "Imagine you are a helpful assistant at Krea University who students come to clarify doubts regarding the university's Data Science curriculum. "
        "Answer the query only based on the context provided. Think step by step before providing a detailed answer. "
        "If you can't answer the query from the context, say that you can't. Do not hallucinate.\n"
        "The CONTEXT is as follows:\n{}"
    )
    formatted_sys_message = SystemMessage(sys_message_template.format(context))

    # Remove any existing SystemMessage from state_messages
    messages = [msg for msg in state_messages if not isinstance(msg, SystemMessage)]

    # Start with the last 'memory_messages' messages from the state
    if len(messages) > memory_messages - 1:
        messages = messages[
            -(memory_messages - 1) :
        ]  # Reserve one spot for SystemMessage

    # Insert the new SystemMessage at the beginning
    messages.insert(0, formatted_sys_message)

    # Add the current user query
    messages.append(HumanMessage(content=query))

    return {"messages": messages}  # returning the prompt

In [22]:
# Function to call Rag Model
def call_rag_model(state: MessagesState):
    response = llm.invoke(state["messages"])

    # Update the state with the new response
    state["messages"].append(response)

    # Keep only the last few messages in the state
    if len(state["messages"]) > memory_messages:
        state["messages"] = state["messages"][
            -1 * (memory_messages) :
        ]  # Update this for more context

    # Maybe limit the context to say 1k tokens?? Maybe use an LLM or to summarise contexts?

    return {"messages": state["messages"]}

In [24]:
# New StageGraph from LangGraph for memory
workflow = StateGraph(state_schema=MessagesState)

workflow.add_edge(START, "model")
workflow.add_node("model", call_rag_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [26]:
# uuid for thread_id for configurable
thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}
# print(thread_id)

In [28]:
# Getting response from llm
def get_response(query, state):
    input_data = query_to_prompt(query, state["messages"])
    output = app.invoke(input_data, config)

    # updating state history
    state["messages"] = output["messages"]

    llm_response = output["messages"][
        -1
    ]  # modified very slightly in streamlit deployment to avoid excess ui-outputs

    return llm_response, state

In [30]:
def print_conversation(query, state):
    llm_response, state = get_response(query, state)

    print(f"QUERY: {query}\n")
    llm_response.pretty_print()

In [32]:
state = {"messages": []}  # Initialising Message State

In [34]:
query = "Hi, I am XYZ"
print_conversation(query, state)

QUERY: Hi, I am XYZ

================================== Ai Message ==================================

Hello XYZ! How can I assist you today regarding the Data Science curriculum at Krea University?


In [35]:
query = "What is my name?"  # Checking Conversational nature
print_conversation(query, state)

QUERY: What is my name?

================================== Ai Message ==================================

I'm sorry, but I can't provide that information. You mentioned your name as XYZ. How can I assist you further regarding the Data Science curriculum?


In [36]:
query = "Describe the Natural Language Processing course"
print_conversation(query, state)

QUERY: Describe the Natural Language Processing course

================================== Ai Message ==================================

The Natural Language Processing (NLP) course at Krea University is a 4-credit course offered in Trimester 10. It provides a theoretical and methodological introduction to NLP, focusing on various aspects such as:

1. **Basic Language Models**: Understanding foundational models used in NLP.
2. **Part-of-Speech (POS) Tagging**: Techniques for identifying the grammatical parts of words in sentences.
3. **Syntactic Parsing**: Analyzing the structure of sentences.
4. **Semantic Analysis**: Understanding the meaning of words and sentences.

The course covers statistical and machine learning approaches, including Hidden Markov Models and Recurrent Neural Networks, for various NLP tasks/applications such as:

- Machine Translation
- Sentiment Analysis
- Question Answering
- Information Extraction

Additionally, the course emphasizes practical implementation 

In [37]:
# Get Questions from Questions.txt document for evaluation